In [26]:
from google.cloud import translate_v2 as translate
import requests
from bs4 import BeautifulSoup
from collections import Counter
import re
import pandas as pd
import os
from function_words import unique_function_words

In [27]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/roneng100/API Keys/website-translation-419023-4572f2415643.json"

In [28]:
def extract_words_from_txt(file_path):
    try:
        # Open the file with default UTF-8 encoding
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        # Try ISO-8859-1 encoding
        with open(file_path, 'r', encoding='ISO-8859-1') as file:
            text = file.read()
    except Exception as e:
        raise e
    
    words = text.split()  # Split text by whitespace

    return words

file_path = 'Subject_Terms.txt'
words = extract_words_from_txt(file_path)
print(words)

['SUBJECT', 'TERMS:', 'FINAL', 'COMMITTEE', 'DRAFT', 'Agriculture', 'and', 'Food', 'Agricultural', 'marketing', 'Agricultural', 'practices', 'Agricultural', 'research', 'Agricultural', 'subsidies', 'Alcoholic', 'beverages', 'Aquaculture', 'Farmland', 'Food', 'assistance', 'Food', 'industry', 'Food', 'labeling', 'Food', 'safety', 'Genetically', 'modified', 'organisms', '(G.M.O.)', 'Livestock', 'Animals', 'Animal', 'diseases', 'Animal', 'protection', 'Service', 'animals', 'Veterinary', 'medicine', 'Wildlife', '{scope', 'note:', 'includes', 'tracking,', 'trafficking,', 'poaching,', 'and', 'refuges}', 'Arts', 'and', 'Culture', '{scope', 'note:', 'See', '�Intellectual', 'Property�', 'for', 'copyright-related', 'matters.}', 'Archives', 'Attire', 'Cultural', 'heritage', 'Libraries', 'Museums', 'Religious', 'buildings', 'Statues', 'and', 'monuments', 'Television', 'and', 'film', 'Commerce', '{scope', 'note:', 'Includes', 'e-commerce;', 'See', 'also', '�Foreign', 'Trade', 'and', 'International'

In [29]:
def clean_text(word_list):
    clean_words = [re.sub(r'[^a-zA-Z]', '', word) for word in word_list] # Remove non-alphabetical characters (keep a-z and A-Z)
    clean_words = [word for word in clean_words if word] # Remove empty string
    return clean_words

cleaned_word = clean_text(words)
print(len(cleaned_word), cleaned_word)
unique_words = set(cleaned_word)
print(len(unique_words), unique_words)
filter_words = list(unique_words - unique_function_words)
print(len(filter_words), filter_words)

1766 ['SUBJECT', 'TERMS', 'FINAL', 'COMMITTEE', 'DRAFT', 'Agriculture', 'and', 'Food', 'Agricultural', 'marketing', 'Agricultural', 'practices', 'Agricultural', 'research', 'Agricultural', 'subsidies', 'Alcoholic', 'beverages', 'Aquaculture', 'Farmland', 'Food', 'assistance', 'Food', 'industry', 'Food', 'labeling', 'Food', 'safety', 'Genetically', 'modified', 'organisms', 'GMO', 'Livestock', 'Animals', 'Animal', 'diseases', 'Animal', 'protection', 'Service', 'animals', 'Veterinary', 'medicine', 'Wildlife', 'scope', 'note', 'includes', 'tracking', 'trafficking', 'poaching', 'and', 'refuges', 'Arts', 'and', 'Culture', 'scope', 'note', 'See', 'Intellectual', 'Property', 'for', 'copyrightrelated', 'matters', 'Archives', 'Attire', 'Cultural', 'heritage', 'Libraries', 'Museums', 'Religious', 'buildings', 'Statues', 'and', 'monuments', 'Television', 'and', 'film', 'Commerce', 'scope', 'note', 'Includes', 'ecommerce', 'See', 'also', 'Foreign', 'Trade', 'and', 'International', 'Finance', 'Build

In [30]:
file_path = './website_links.csv'

def read_urls_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        urls = [line.strip() for line in file]
    return urls

urls_list = read_urls_from_file(file_path)
print(urls_list)

['https://boe.es/boe/dias/2005/12/30/', 'https://boe.es/boe/dias/2005/12/31/', 'https://boe.es/boe/dias/2006/01/03/', 'https://boe.es/boe/dias/2006/01/10/', 'https://boe.es/boe/dias/2006/01/17/', 'https://boe.es/boe/dias/2006/01/18/', 'https://boe.es/boe/dias/2006/01/20/', 'https://boe.es/boe/dias/2006/01/24/', 'https://boe.es/boe/dias/2006/01/25/', 'https://boe.es/boe/dias/2006/01/30/', 'https://boe.es/boe/dias/2006/02/02/', 'https://boe.es/boe/dias/2006/02/03/', 'https://boe.es/boe/dias/2006/02/07/', 'https://boe.es/boe/dias/2006/02/09/', 'https://boe.es/boe/dias/2006/02/10/', 'https://boe.es/boe/dias/2006/02/11/', 'https://boe.es/boe/dias/2006/02/16/', 'https://boe.es/boe/dias/2006/02/17/', 'https://boe.es/boe/dias/2006/02/18/', 'https://boe.es/boe/dias/2006/02/20/', 'https://boe.es/boe/dias/2006/02/23/', 'https://boe.es/boe/dias/2006/02/24/', 'https://boe.es/boe/dias/2006/02/25/', 'https://boe.es/boe/dias/2006/02/28/', 'https://boe.es/boe/dias/2006/03/02/', 'https://boe.es/boe/dias

In [31]:
def translate_text_chunks(translator, text, target_language='en'):
    MAX_LENGTH = 102400  # Define the maximum length of text per API request
    chunks = [text[i:i + MAX_LENGTH] for i in range(0, len(text), MAX_LENGTH)]
    
    translated_text = ''
    for chunk in chunks:
        try:
            response = translator.translate(chunk, target_language=target_language)
            translated_text += response['translatedText'].lower()
        except Exception as e:
            # Handle translation errors
            translated_text += f" [Error translating chunk: {str(e)}]"
    
    return translated_text

def check_words_on_webpages(urls, unique_words):
    website_word_presence = {}
    translator = translate.Client()

    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                text = soup.get_text().lower()

                # Translate the text from Spanish to English
                translated_text = translate_text_chunks(translator, text, target_language='en')
                # Clean the text
                clean_text = re.sub('[^a-z\s]', '', translated_text)
                words_present = set(clean_text.split())

                # Filter the words present in the unique_words list
                found_words = [word for word in unique_words if word in words_present]
                
                website_word_presence[url] = found_words
            else:
                website_word_presence[url] = f"Skipping URL due to non-200 status code: {response.status_code}"
        except requests.RequestException as e:
            website_word_presence[url] = f"Skipping URL due to request exception: {str(e)}"

    return website_word_presence

# Example usage:
urls = ['https://boe.es/boe/dias/2005/12/30/']
found_words = check_words_on_webpages(urls, filter_words)
print(found_words)

{'https://boe.es/boe/dias/2005/12/30/': ['highways', 'vehicles', 'transportation', 'employees', 'air', 'systems', 'schools', 'language', 'management', 'other', 'occupation', 'aid', 'policy', 'public', 'markets', 'commercial', 'airports', 'materials', 'plant', 'political', 'training', 'law', 'treatment', 'conservation', 'hiring', 'court', 'buildings', 'international', 'local', 'called', 'contracts', 'protection', 'quality', 'devices', 'disabilities', 'proceedings', 'areas', 'community', 'transfer', 'state', 'safety', 'trade', 'bodies', 'activities', 'navigation', 'products', 'agencies', 'medical', 'works', 'legal', 'reform', 'administration', 'development', 'matters', 'health', 'exchange', 'information', 'gas', 'organizations', 'industry', 'support', 'tax', 'national', 'scope', 'credits', 'water', 'financial', 'fees', 'work', 'control', 'canals', 'general', 'surveillance', 'security', 'utility', 'construction', 'resolution', 'education', 'services', 'terms', 'assistance', 'order', 'stat

In [32]:
# Parameters
max_cols = 20  # Maximum number of columns for words

# Function to prepare data for DataFrame
def prepare_data(data, max_cols):
    rows = []
    merge_info = []
    for url, content in data.items():
        if isinstance(content, list):  # Check if the content is a list of words
            num_rows = -(-len(content) // max_cols)  # Ceiling division for rows
            merge_info.append((url, num_rows))

            # Split words into the appropriate number of rows
            for start in range(0, len(content), max_cols):
                end = start + max_cols
                row = [url] + content[start:end]
                row += [None] * (max_cols - len(content[start:end]))  # Correct padding calculation
                rows.append(row)
        else:
            # Handle error messages
            row = [url] + [content] + [None] * (max_cols - 1)  # Correct number of None to fill up the row
            rows.append(row)
            merge_info.append((url, 1))  # Only one row for error messages

    return rows, merge_info


In [33]:
# Prepare rows and merging information
data = check_words_on_webpages(urls_list, filter_words)
data_rows, merge_info = prepare_data(data, max_cols)

# Create DataFrame
column_names = ['URL'] + [f"Word_{i+1}" for i in range(max_cols)]
df = pd.DataFrame(data_rows, columns=column_names)

# Export to Excel with merged cells and formatted headers
with pd.ExcelWriter('URL_Subject_Terms.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Formatting for headers and cells
    header_format = workbook.add_format({'bold': True, 'align': 'left', 'valign': 'top'})
    text_format = workbook.add_format({'bold': True, 'align': 'center', 'valign': 'vcenter', 'font_color': 'black'})

    # Merge the word headers into one cell and name it "Unique Words"
    worksheet.merge_range(0, 1, 0, max_cols, 'Unique Words', header_format)

    # Apply formatting to URLs to prevent Excel from auto-formatting as hyperlinks
    for index, _ in enumerate(data_rows):
        worksheet.write_string(index + 1, 0, data_rows[index][0], text_format)  # +1 for header row

    # Apply merging for URL column based on the number of rows each URL occupies
    current_row = 1  # Excel rows start from 1 and account for the header
    for url, rows in merge_info:
        # Only apply the merging if more than one row is needed for the URL
        if rows > 1:
            worksheet.merge_range(current_row, 0, current_row + rows - 1, 0, url, text_format)
        current_row += rows

    # Set column widths
    worksheet.set_column(0, 0, 20)  # URL column wider
    for i in range(max_cols):
        worksheet.set_column(i+1, i+1, 15)  # Word columns